# Student Course Report Analysis
_THIS IS NOT USED, I ENDED UP DOING EVERYTHING IN JS ON THE DASHBOARD PAGE_

In [27]:
# Imports
import pandas as pd
import os
import re
import json
import matplotlib.pyplot as plt
import functools

from scipy import stats
from os.path import join, dirname
from pymongo import MongoClient
from pprint import pprint
from dotenv import load_dotenv
from IPython.display import display, JSON, HTML

# Load environment variables
load_dotenv(join(os.path.abspath(""), "../.env.local"))

pass

In [28]:
# Connect to database
mongo_client = MongoClient(os.environ.get("MONGO_URI"))
db = mongo_client["george-beta"]

# Load data
professors = pd.DataFrame(db["professors"].find({}))
courses = pd.DataFrame(db["courses"].find({}))
sections = pd.DataFrame(db["sections"].find({}))

In [29]:
# The question text for each number
question_texts = {
    1: "My overall rating of the quality of this course is",
    2: "My overall rating of the instructor's teaching is",
    3: "The educational value of the assigned work was",
    4: "The instructor's organization of the course was",
    5: "The instructor's clarity in communicating course objectives was",
    6: "The instructor's skill in providing understandable explanations was",
    7: "The amount I learned from the course was",
    8: "The intellectual challenge presented by the course was",
    9: "The instructor's personal interest in helping students learn was",
    10: "The instructor stimulated my interest in the subject matter",
    11: "The amount of reading, homework, and other assigned work was",
    12: "The instructor was well prepared to teach class.",
    13: "The instructor encouraged students to ask questions.",
    14: "The instructor treated students with respect.",
    15: "Instructor feedback on exams/assignments was timely and helpful.",
    16: "The exams and/or evaluations were good measures of the material covered.",
    17: "My grades were determined in a fair and impartial manner.",
    18: "What grade do you think you will receive in this course?",
    19: "On average, what were the total hours spent in each 7-day week OUTSIDE of formally scheduled class time in work related to this course (including studying, reading, writing, homework, rehearsal, etc.)?"
}

# Categories of questions (subjective)
question_categories = {
    "course_quality": [1, 2, 3, 4, 5, 6],
    "course_difficulty": [7, 8, 9, 10, 11, 19],
    "instructor_quality": [12, 13, 14, 15, 16],
    "grades": [17, 18]
}

In [30]:
# Augment the data

# Assign professors to 1+ department


# Calculate the average response of every question in a category
def add_category(row, questions):
    category_sum = 0
    for question in row.report:
        if question["questionNumber"] not in questions:
            continue
        #print(question["questionNumber"])
        responses = question["responses"]
        weighted_responses = [x*(i+1) for i, x in enumerate(responses)]

        if sum(responses) == 0:
            return None
        else:
            mean_response = sum(weighted_responses) / sum(responses)

        category_sum += mean_response
        #print(responses, weighted_responses, mean_response)
    
    category_mean = category_sum / len(questions)
    
    return category_mean

for category, questions in question_categories.items():
    sections[category] = sections.apply(lambda row: add_category(row, questions), axis=1)

In [ ]:
# Filter sections
def get_sections_by_department(sections, dept):
    return sections[sections.apply(lambda r: re.search(f"^{dept}", r["courseId"]) is not None, axis=1 )]

def get_sections_by_professor(sections, professor_id): 
    return sections[sections["professorId"] == professor_id]

In [31]:
random_prof = "935932"
#display(get_sections_by_professor(random_prof))

,_id,professorId,professorName,courseId,courseName,term,sectionType,report,course_quality,course_difficulty,instructor_quality,grades
570,625d9218cf1c6744ae881445,935932,"Wong, Wilson",CS 3431,DATABASE SYSTEMS I,201901_A,Lecture,"[{'questionNumber': 1, 'responses': [0, 3, 0, ...",4.468231,3.781565,4.646694,3.230867
575,625d921acf1c6744ae88144a,935932,"Wong, Wilson",CS 3733,SOFTWARE ENGINEERING,201902_C,Lab,"[{'questionNumber': 1, 'responses': [1, 2, 3, ...",4.069444,4.279167,4.109848,3.088768
576,625d921bcf1c6744ae88144b,935932,"Wong, Wilson",CS 3733,SOFTWARE ENGINEERING,201902_D,Lab,"[{'questionNumber': 1, 'responses': [0, 3, 4, ...",4.556498,4.627803,4.425147,3.126866
2347,625d95d9cf1c6744ae881b36,935932,"Wong, Wilson",CS 3431,DATABASE SYSTEMS I,202001_A,Lecture,"[{'questionNumber': 1, 'responses': [1, 0, 1, ...",4.646465,4.074021,4.706250,3.451613
2352,625d95dbcf1c6744ae881b3b,935932,"Wong, Wilson",CS 3733,SOFTWARE ENGINEERING,202002_C,Lab,"[{'questionNumber': 1, 'responses': [0, 0, 1, ...",4.811111,4.718750,4.575000,3.187500
3889,625d996acf1c6744ae88213c,935932,"Wong, Wilson",CS 3431,DATABASE SYSTEMS I,202101_A,Lecture,"[{'questionNumber': 1, 'responses': [0, 0, 0, ...",4.791667,4.029943,4.786882,3.261593
3894,625d996ccf1c6744ae882141,935932,"Wong, Wilson",CS 3733,SOFTWARE ENGINEERING,202102_C,Lab,"[{'questionNumber': 1, 'responses': [1, 2, 3, ...",3.985507,NaN,4.038735,2.739130
3895,625d996dcf1c6744ae882142,935932,"Wong, Wilson",CS 3733,SOFTWARE ENGINEERING,202102_D,Lab,"[{'questionNumber': 1, 'responses': [2, 5, 3, ...",3.961081,4.399049,4.200000,3.424242
6184,625d9dfdcf1c6744ae882a33,935932,"Wong, Wilson",CS 3431,Database Systems I,202201_A,Lecture,"[{'questionNumber': 1, 'responses': [0, 0, 1, ...",4.676861,4.105005,4.836495,3.227345
6188,625d9dffcf1c6744ae882a37,935932,"Wong, Wilson",CS 3733,Software Engineering,202202_C,Lab,"[{'questionNumber': 1, 'responses': [1, 2, 1, ...",4.133333,4.580263,4.310000,3.048246
